In [7]:
import json
import pickle
import numpy as np
from sklearn.preprocessing import LabelEncoder

IMG_FEAT_PATH = '../data/simaese_img_feats/simaese_mob_299_sim_hbp_best_train_loss.pkl'

fin = open(IMG_FEAT_PATH, 'rb')
train_feats = pickle.load(fin)
fin.close()
print(train_feats[0][0], train_feats[0][1], train_feats[0][2].shape)

w_b938e96 ../data/train/054bb95c0.jpg (256,)


In [8]:
# only keep label cnt >= 2
x_data, y_data = [], []
train_cnt = {}
for label, _, _ in train_feats:
    if label not in train_cnt:
        train_cnt[label] = 0
    train_cnt[label] += 1

filter_labels = set()
for label in train_cnt:
    if train_cnt[label] >=4 :
        filter_labels.add(label)
print('left labels', len(filter_labels))

for label, _, feat in train_feats:
    if label in filter_labels:
        x_data.append(feat)
        y_data.append(label)

le = LabelEncoder()
y_l_data = le.fit_transform(y_data)
x_data = np.array(x_data)
print(x_data.shape, y_l_data.shape)

left labels 1078
(9350, 256) (9350,)


In [9]:
print(le.classes_)
print(len(le.classes_))

['w_0027efa' 'w_003bae6' 'w_007fefa' ... 'w_fec331a' 'w_fec5547'
 'w_ff2157c']
1078


In [10]:
# cv check
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LogisticRegression

X_train, X_test, y_train, y_test = train_test_split(
    x_data, y_l_data, test_size=0.33, random_state=42)
lr_model = LogisticRegression(
    random_state=0, solver='lbfgs', multi_class='multinomial')
lr_model.fit(x_data, y_l_data)
print('fit done')

fit done


In [11]:
print('all data score', lr_model.score(x_data, y_l_data))
print('test data score', lr_model.score(X_test, y_test))
all_pred = lr_model.predict_proba(x_data)
test_pred = lr_model.predict_proba(X_test)
print(test_pred[0], sum(test_pred[0]))


def check_topk_acc(y_true, y_pred, k=3):
    correct = 0
    for i, tmp_pred in enumerate(y_pred):
        topk = tmp_pred.argsort()[-k:]
        # print(topk,y_true[i])
        if y_true[i] in topk:
            correct += 1
    print(correct*1.0/len(y_true))


print('check topk for distribution')
check_topk_acc(y_test, test_pred, 3)
check_topk_acc(y_test, test_pred, 4)
check_topk_acc(y_test, test_pred, 5) 

# label_cnt >=4 
# all data score 0.2474866310160428
# test data score 0.2524303305249514
# [4.35346186e-03 6.53737412e-03 6.97298824e-06 ... 3.42658140e-03
#  2.60473778e-03 2.40953454e-04] 0.9999999999999992
# check topk for distribution
# 0.41088788075178223
# 0.46208684381075826
# 0.5100453661697991

# label_cnt >= 3
# all data score 0.21024063687352995
# test data score 0.21710526315789475
# [5.15169970e-06 1.66555342e-05 1.66082321e-06 ... 1.15979089e-06
#  8.44756817e-04 3.39109691e-03] 1.0000000000000009
# check topk for distribution
# 0.3706140350877193
# 0.4161184210526316
# 0.45586622807017546


all data score 0.2474866310160428
test data score 0.2524303305249514
[4.35346186e-03 6.53737412e-03 6.97298824e-06 ... 3.42658140e-03
 2.60473778e-03 2.40953454e-04] 0.9999999999999992
check topk for distribution
0.41088788075178223
0.46208684381075826
0.5100453661697991


In [12]:
with open('../data/checkpoints/lr_top3_model.pkl', 'wb') as fout:
    pickle.dump(lr_model, fout)
print('save done')

save done
